# Patient similarity network

In [2]:
# standard libraries
import os
import pickle

# scientific and data manipulation libraries
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.feature_selection import mutual_info_regression
from astropy.stats import median_absolute_deviation
import mygene
import astropy

# graph and network libraries
import networkx as nx
from pyvis.network import Network

# visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.io as pio

- Based on the same expression matrix we can create a patient similarity network.
- By transposing the matrix will switch the rows and columns,
- meaning that patients will become the rows and genes will become the columns.
- By doing this, you can compute the correlation (or similarity) between patients based on their gene expression profiles,
- and then create a network where nodes represent patients and edges represent similarities.

In [3]:
# main data directory for the project

data_dir = '../../ismb_data'

In [4]:
# read in os.path.join(data_dir,"expression_data_filtered.csv")
df_renamed = pd.read_csv(os.path.join(data_dir,"expression_data_filtered.csv"),index_col=0)

In [5]:
# Transpose the matrix
patient_gene_matrix = df_renamed.T

In [6]:
patient_gene_matrix

,TCGA-38-7271,TCGA-55-7914,TCGA-95-7043,TCGA-73-4658,TCGA-86-8076,TCGA-55-7726,TCGA-44-6147,TCGA-50-5932,TCGA-44-2661,TCGA-86-7954,...,TCGA-97-A4M7,TCGA-62-A46R,TCGA-50-5055,TCGA-38-4628,TCGA-86-7713,TCGA-86-8073,TCGA-MN-A4N4,TCGA-53-7626,TCGA-44-A47G,TCGA-55-6969
A2M,17.7492,14.8513,14.1691,16.7238,15.6783,14.7566,16.4368,15.5476,15.5478,15.1337,...,15.9553,13.9511,16.3097,14.3934,15.8254,16.3773,14.9411,16.7343,15.6622,14.8136
A2ML1,4.4411,4.4530,4.5026,3.1704,4.7422,6.0918,4.0602,3.0827,3.4608,3.6367,...,4.7076,2.4821,8.0810,3.3415,5.3379,4.0817,6.7210,5.2575,3.7345,4.3058
A4GALT,10.1862,8.9312,9.0834,9.1443,9.5150,11.4452,9.1462,7.3597,9.1602,8.9344,...,9.1019,8.6653,10.8837,8.6304,9.3827,9.4234,11.9107,9.5379,9.8660,9.7702
AACSP1,3.5845,3.6762,3.8623,2.4821,3.2271,2.4821,5.6614,3.0827,3.9428,4.7089,...,5.4947,3.0956,2.4821,3.7710,2.9397,6.6519,8.8283,2.4821,4.1045,6.8425
AADAC,9.6415,6.5685,5.5634,4.3206,7.2321,6.8821,8.1176,4.7935,11.7750,7.0644,...,9.0856,3.8162,5.6471,4.6785,4.6578,6.7641,3.4947,5.7120,6.2214,5.2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZSCAN23,5.7385,4.3796,5.6728,3.6533,5.0478,6.0489,5.8058,3.7897,4.3451,5.5259,...,4.7076,2.4821,4.6683,3.4697,4.3516,7.2074,6.9167,6.0927,5.3854,3.6792
ZSCAN31,9.0817,10.9565,10.8095,8.9874,9.5810,9.0185,10.2085,9.8803,11.0327,10.2707,...,9.2033,10.5020,8.8073,9.8672,10.6419,10.2410,9.4672,10.0507,7.4843,8.5268
ZSCAN4,3.2709,3.0918,2.4821,3.4468,3.0118,5.0894,5.5341,3.5082,5.4987,3.3645,...,4.4088,7.4937,4.3167,5.0907,6.7676,5.1126,3.3143,4.2816,4.4650,2.4821
ZWINT,9.2615,10.1334,10.9149,9.7828,8.6640,11.1212,9.0718,10.7305,9.0013,10.4268,...,9.7614,10.7593,9.7087,10.4729,10.9794,10.1390,10.0583,9.4627,9.5817,10.8228


In [7]:
# Dictionary to store different correlation matrices
patient_correlation_matrices = {}

# Pearson correlation
patient_correlation_matrices['pearson'] = patient_gene_matrix.corr(method='pearson')

# patient_correlation_matrices['biweight_midcorrelation'] = abs_bicorr(patient_gene_matrix)

In [8]:
def create_graph_from_correlation(correlation_matrix, threshold=0.8):
    """
    Creates a graph from a correlation matrix using a specified threshold.

    Parameters:
    correlation_matrix (pd.DataFrame): DataFrame containing the correlation matrix.
    threshold (float): Threshold for including edges based on correlation value.

    Returns:
    G (nx.Graph): Graph created from the correlation matrix.
    """
    G = nx.Graph()

    # Add nodes
    for node in correlation_matrix.columns:
        G.add_node(node)

    # Add edges with weights above the threshold
    for i in range(correlation_matrix.shape[0]):
        for j in range(i + 1, correlation_matrix.shape[1]):
            if i != j:  # Ignore the diagonal elements
                weight = correlation_matrix.iloc[i, j]
                if abs(weight) >= threshold:
                    G.add_edge(correlation_matrix.index[i], correlation_matrix.columns[j], weight=weight)

    return G


def clean_graph(G, degree_threshold=1, keep_largest_component=True):
    """
    Cleans the graph by performing several cleaning steps:
    - Removes unconnected nodes (isolates)
    - Removes self-loops
    - Removes nodes with a degree below a specified threshold
    - Keeps only the largest connected component (optional)

    Parameters:
    G (nx.Graph): The NetworkX graph to clean.
    degree_threshold (int): Minimum degree for nodes to keep.
    keep_largest_component (bool): Whether to keep only the largest connected component.

    Returns:
    G (nx.Graph): Cleaned graph.
    """
    G = G.copy()  # Work on a copy of the graph to avoid modifying the original graph

    # Remove self-loops
    G.remove_edges_from(nx.selfloop_edges(G))

    # Remove nodes with no edges (isolates)
    G.remove_nodes_from(list(nx.isolates(G)))

    # Remove nodes with degree below the threshold
    low_degree_nodes = [node for node, degree in dict(G.degree()).items() if degree < degree_threshold]
    G.remove_nodes_from(low_degree_nodes)

    # Keep only the largest connected component
    if keep_largest_component:
        largest_cc = max(nx.connected_components(G), key=len)
        G = G.subgraph(largest_cc).copy()

    return G

def print_graph_info(G):
    """
    Prints basic information about the graph.
    
    Parameters:
    G (nx.Graph): The NetworkX graph.
    """
    print(f"Number of nodes: {G.number_of_nodes()}")
    print(f"Number of edges: {G.number_of_edges()}")
    print("Sample nodes:", list(G.nodes)[:10])  # Print first 10 nodes as a sample
    print("Sample edges:", list(G.edges(data=True))[:10])  # Print first 10 edges as a sample
    
    # Check for self-loops
    self_loops = list(nx.selfloop_edges(G))
    if self_loops:
        print(f"Number of self-loops: {len(self_loops)}")
        print("Self-loops:", self_loops)
    else:
        print("No self-loops in the graph.")

def calculate_average_clustering(G):
    """
    Calculates and prints the average clustering coefficient of the graph.
    
    Parameters:
    G (nx.Graph): The NetworkX graph.
    """
    avg_clustering = nx.average_clustering(G)
    print(f"Average clustering coefficient: {avg_clustering}")





def knn_sparsification(graph, k):
    """
    Sparsifies the graph by keeping only the top-k edges with the highest weights for each node.

    Parameters:
    graph (nx.Graph): The original NetworkX graph.
    k (int): The number of nearest neighbors to keep for each node.

    Returns:
    nx.Graph: The sparsified graph.
    """
    graph_copy = graph.copy()
    sparsified_graph = nx.Graph()
    sparsified_graph.add_nodes_from(graph_copy.nodes(data=True))
    
    for node in graph_copy.nodes():
        edges = sorted(graph_copy.edges(node, data=True), key=lambda x: x[2].get('weight', 0), reverse=True)
        sparsified_graph.add_edges_from(edges[:k])
    
    return sparsified_graph

In [ ]:
patient_pearson_graph = create_graph_from_correlation(patient_correlation_matrices['pearson'], threshold=0.8)

In [ ]:
visualize_graph(patient_pearson_graph, title='Pearson Correlation Network (Threshold = 0.8)')

In [ ]:
# Clean the graph by removing unconnected nodes
patient_pearson_graph_pruned = clean_graph(patient_pearson_graph,
                                    degree_threshold=1,
                                    keep_largest_component=True)

In [ ]:
visualize_graph(patient_pearson_graph_pruned, title='Pearson Patient Correlation Network (Threshold = 0.8)')

In [ ]:
patient_pearson_graph_pruned_knn = knn_sparsification(patient_pearson_graph_pruned, k=10)

In [ ]:
print_graph_info(patient_pearson_graph_pruned)

In [ ]:
print_graph_info(patient_pearson_graph_pruned_knn)

In [ ]:
visualize_graph(patient_pearson_graph_pruned_knn, title='K-Nearest Neighbors (k=10) Patient Correlation Network')

In [ ]:
nx.write_gml(patient_pearson_graph_pruned_knn, os.path.join(data_dir,'patient_coexpression_network_knn.gml'))

In [22]:
def save_graphs(correlation_matrices_dict, data_dir, matrix_names):
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    # Define parameter combinations
    thresholds = [0.6, 0.7, 0.8, 0.9]
    k_values = [3, 5, 7, 10]
    degree_thresholds = [0, 1]
    keep_largest_component_options = [False, True]

    for matrix_name in matrix_names:
        if matrix_name not in correlation_matrices_dict:
            print(f"Matrix {matrix_name} not found in the provided dictionary. Skipping...")
            continue
        
        correlation_matrix = correlation_matrices_dict[matrix_name]
        
        for threshold in thresholds:
            # Create the initial graph from the correlation matrix
            initial_graph = create_graph_from_correlation(correlation_matrix, threshold=threshold)
            
            for degree_threshold in degree_thresholds:
                for keep_largest_component in keep_largest_component_options:
                    # Clean the graph
                    cleaned_graph = clean_graph(initial_graph, degree_threshold=degree_threshold, keep_largest_component=keep_largest_component)
                    
                    for k in k_values:
                        # Sparsify the graph using KNN
                        sparsified_graph = knn_sparsification(cleaned_graph, k)
                        
                        # Calculate the average clustering coefficient
                        avg_clustering = nx.average_clustering(sparsified_graph)
                        
                        # Construct the filename with relevant information
                        filename = f"{matrix_name}_network_threshold_{threshold}_k_{k}_degree_{degree_threshold}_largest_{keep_largest_component}_clustering_{avg_clustering:.2f}.gml"
                        filepath = os.path.join(data_dir, filename)
                        
                        # Save the graph to a GML file
                        nx.write_gml(sparsified_graph, filepath)
                        
                        # Print the details
                        print(f"Saved: {filepath}")

def main(correlation_matrices_dict, matrix_names):
    data_dir = '../../ismb_data/05072024/PSN-Met'  # Adjust the directory path as needed
    save_graphs(correlation_matrices_dict, data_dir, matrix_names)

if __name__ == "__main__":
    # correlation_matrices_dict = {
    #     'pearson': pd.DataFrame(),  # Replace with actual data
    #     'spearman': pd.DataFrame()  # Replace with actual data if needed
    # }
    matrix_names = ['pearson']  # Replace with the desired correlation matrix names
    main(p_meth_correlation_matrices, matrix_names)

Saved: ../../ismb_data/05072024/PSN-Met/pearson_network_threshold_0.6_k_3_degree_0_largest_False_clustering_0.21.gml
Saved: ../../ismb_data/05072024/PSN-Met/pearson_network_threshold_0.6_k_5_degree_0_largest_False_clustering_0.25.gml
Saved: ../../ismb_data/05072024/PSN-Met/pearson_network_threshold_0.6_k_7_degree_0_largest_False_clustering_0.27.gml
Saved: ../../ismb_data/05072024/PSN-Met/pearson_network_threshold_0.6_k_10_degree_0_largest_False_clustering_0.30.gml
Saved: ../../ismb_data/05072024/PSN-Met/pearson_network_threshold_0.6_k_3_degree_0_largest_True_clustering_0.21.gml
Saved: ../../ismb_data/05072024/PSN-Met/pearson_network_threshold_0.6_k_5_degree_0_largest_True_clustering_0.25.gml
Saved: ../../ismb_data/05072024/PSN-Met/pearson_network_threshold_0.6_k_7_degree_0_largest_True_clustering_0.27.gml
Saved: ../../ismb_data/05072024/PSN-Met/pearson_network_threshold_0.6_k_10_degree_0_largest_True_clustering_0.30.gml
Saved: ../../ismb_data/05072024/PSN-Met/pearson_network_threshold_

# DNAmeth PN

In [12]:
# Load the data using pickle
with open(os.path.join(data_dir,"ISMB_TCGA_DNAm.pkl") , 'rb') as file : 
    data = pd.read_pickle(file)

meth_data = data["datExpr"]
meth_data


,cg23057992,cg16602369,cg20545544,cg04571941,cg23091104,cg11738485,cg17735539,cg12662576,cg10554839,cg08866780,...,cg04339424,cg01318691,cg27068965,cg23028848,cg21122529,cg22254072,cg10237911,cg09874052,cg27624178,cg02857943
TCGA-55-7914,0.039144,0.078881,0.032201,0.098520,0.971626,0.020020,0.929670,0.033325,0.043767,0.028051,...,0.053027,0.046047,0.951328,0.045767,0.048185,0.014262,0.034541,0.034967,0.041538,0.053116
TCGA-38-4631,0.954805,0.047287,0.027976,0.925794,0.144636,0.365842,0.156414,0.027917,0.047609,0.049901,...,0.048398,0.047221,0.939891,0.056141,0.047459,0.014548,0.056591,0.039086,0.044946,0.055473
TCGA-73-4658,0.966071,0.126549,0.030316,0.945166,0.976080,0.975638,0.942018,0.066659,0.052100,0.047054,...,0.040815,0.042040,0.951235,0.045477,0.074037,0.019053,0.029978,0.039345,0.060650,0.041385
TCGA-50-5932,0.965140,0.061194,0.935457,0.923421,0.965443,0.027598,0.040885,0.036178,0.045745,0.046439,...,0.057840,0.045106,0.948255,0.044558,0.039484,0.013837,0.037048,0.032503,0.058576,0.037666
TCGA-55-7576,0.958612,0.071607,0.027077,0.940574,0.976870,0.785073,0.953674,0.945107,0.056088,0.955472,...,0.046921,0.051442,0.954000,0.034352,0.031497,0.014924,0.035520,0.043547,0.056388,0.052208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-50-8457,0.960316,0.940219,0.046927,0.942980,0.977751,0.633731,0.940723,0.053398,0.059292,0.968875,...,0.047640,0.062680,0.954824,0.043137,0.048599,0.014066,0.031109,0.039247,0.035110,0.054273
TCGA-91-6840,0.964250,0.921250,0.957943,0.958416,0.981439,0.278432,0.968525,0.032143,0.083996,0.037912,...,0.045197,0.066271,0.963420,0.047621,0.060854,0.015823,0.045674,0.040006,0.052008,0.040826
TCGA-86-8280,0.948855,0.942925,0.958506,0.945667,0.970511,0.544447,0.921924,0.939427,0.082121,0.045036,...,0.062367,0.058898,0.954355,0.045702,0.048930,0.019321,0.039341,0.036627,0.054269,0.056795
TCGA-95-A4VK,0.037255,0.050940,0.036565,0.098237,0.976780,0.683394,0.960073,0.025232,0.062487,0.026908,...,0.056248,0.053591,0.947020,0.035344,0.057783,0.010943,0.051868,0.035566,0.044910,0.042269


In [13]:
# load the data from the pickle file
with open(os.path.join(data_dir,"ISMB_TCGA_GE.pkl"), 'rb') as file:
    GE_data = pickle.load(file)


In [15]:
data["datMeta"]["patient"].to_list()

['TCGA-55-7914',
 'TCGA-38-4631',
 'TCGA-73-4658',
 'TCGA-50-5932',
 'TCGA-55-7576',
 'TCGA-55-8615',
 'TCGA-44-2655',
 'TCGA-50-5939',
 'TCGA-86-7954',
 'TCGA-44-7671',
 'TCGA-78-8660',
 'TCGA-62-A46P',
 'TCGA-55-6978',
 'TCGA-86-8674',
 'TCGA-55-A48Y',
 'TCGA-78-8662',
 'TCGA-50-5931',
 'TCGA-55-8299',
 'TCGA-62-8398',
 'TCGA-44-5643',
 'TCGA-69-7763',
 'TCGA-86-8279',
 'TCGA-75-6214',
 'TCGA-93-8067',
 'TCGA-97-8179',
 'TCGA-50-5944',
 'TCGA-55-A48Z',
 'TCGA-55-8089',
 'TCGA-L9-A5IP',
 'TCGA-MN-A4N5',
 'TCGA-38-A44F',
 'TCGA-MP-A4SW',
 'TCGA-97-A4M6',
 'TCGA-86-8055',
 'TCGA-91-6835',
 'TCGA-55-A48X',
 'TCGA-44-6774',
 'TCGA-50-5066',
 'TCGA-95-A4VN',
 'TCGA-50-5930',
 'TCGA-49-AAR0',
 'TCGA-44-7672',
 'TCGA-44-2656',
 'TCGA-O1-A52J',
 'TCGA-62-8402',
 'TCGA-55-6642',
 'TCGA-69-8253',
 'TCGA-44-7669',
 'TCGA-86-8358',
 'TCGA-L9-A444',
 'TCGA-49-6761',
 'TCGA-NJ-A4YQ',
 'TCGA-53-7624',
 'TCGA-78-7143',
 'TCGA-L9-A7SV',
 'TCGA-97-A4M3',
 'TCGA-35-5375',
 'TCGA-78-7163',
 'TCGA-78-8648

In [18]:
smoking_df = pd.read_csv(os.path.join(data_dir,"smoking.tsv") , delimiter='\t') #Tab seperated document from EWAS
common_annotated_cpgs = list(smoking_df['cpg'].value_counts()[smoking_df['cpg'].value_counts() > 10].index) 
cpgs = set(common_annotated_cpgs) & set(meth_data.columns)


common_patients = set(data["datMeta"]["patient"].to_list()) & set(meth_data.index)


meth_data_filt = meth_data.loc[ list(common_patients) , list(cpgs)]

patient_meth_matrix = meth_data_filt.T


In [19]:
patient_meth_matrix 


,TCGA-05-4396,TCGA-78-7155,TCGA-44-7659,TCGA-44-A4SU,TCGA-95-7948,TCGA-NJ-A4YF,TCGA-62-A46R,TCGA-86-8279,TCGA-91-6836,TCGA-69-8253,...,TCGA-64-5775,TCGA-55-7910,TCGA-55-6969,TCGA-49-AAQV,TCGA-97-8552,TCGA-MP-A4T4,TCGA-78-7160,TCGA-97-A4M3,TCGA-50-6595,TCGA-55-6982
cg17287155,0.958676,0.566635,0.954226,0.940611,0.968043,0.946452,0.860949,0.953696,0.973568,0.916756,...,0.695966,0.962198,0.576167,0.951114,0.965335,0.918081,0.935858,0.960996,0.963649,0.977802
cg08035323,0.204264,0.422195,0.346077,0.158932,0.187505,0.432249,0.357338,0.274765,0.678613,0.251432,...,0.106467,0.605801,0.426653,0.218842,0.149895,0.180247,0.227759,0.415433,0.229493,0.224023
cg19717773,0.853913,0.958796,0.948303,0.897569,0.953298,0.941726,0.871651,0.878930,0.822781,0.929314,...,0.874589,0.435955,0.596656,0.827823,0.832132,0.804611,0.776288,0.890808,0.650614,0.638636
cg17924476,0.578420,0.178692,0.667851,0.473261,0.622205,0.648433,0.679621,0.326106,0.779092,0.428426,...,0.306427,0.710742,0.406598,0.426056,0.513044,0.486175,0.527042,0.487867,0.475064,0.605959
cg27241845,0.190956,0.107218,0.271998,0.326582,0.210409,0.292984,0.323971,0.170928,0.195694,0.246759,...,0.201612,0.431354,0.415353,0.536400,0.376193,0.485102,0.393647,0.221291,0.505748,0.361600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cg26361535,0.707913,0.898110,0.719424,0.560542,0.847301,0.742228,0.810642,0.741395,0.802132,0.706401,...,0.812918,0.743279,0.788556,0.692957,0.713474,0.758808,0.706660,0.750235,0.664126,0.690027
cg06235438,0.805387,0.897500,0.781594,0.795184,0.886040,0.844269,0.691382,0.834697,0.919788,0.778454,...,0.728339,0.917023,0.614139,0.648874,0.799862,0.634457,0.625825,0.834810,0.807956,0.765255
cg00310412,0.324739,0.254853,0.378588,0.430408,0.676291,0.357737,0.549030,0.376939,0.558502,0.484490,...,0.272397,0.261938,0.425177,0.510116,0.545462,0.411286,0.463077,0.489310,0.466666,0.457315
cg11231349,0.295414,0.781144,0.560626,0.431219,0.836589,0.766791,0.801446,0.463695,0.705177,0.434514,...,0.189563,0.688879,0.677946,0.487507,0.566952,0.605613,0.650328,0.548267,0.628291,0.680081


In [20]:
# Dictionary to store different correlation matrices
p_meth_correlation_matrices = {}

# Pearson correlation
p_meth_correlation_matrices['pearson'] = patient_meth_matrix.corr(method='pearson')


In [21]:
p_meth_correlation_matrices['pearson']

,TCGA-05-4396,TCGA-78-7155,TCGA-44-7659,TCGA-44-A4SU,TCGA-95-7948,TCGA-NJ-A4YF,TCGA-62-A46R,TCGA-86-8279,TCGA-91-6836,TCGA-69-8253,...,TCGA-64-5775,TCGA-55-7910,TCGA-55-6969,TCGA-49-AAQV,TCGA-97-8552,TCGA-MP-A4T4,TCGA-78-7160,TCGA-97-A4M3,TCGA-50-6595,TCGA-55-6982
TCGA-05-4396,1.000000,0.193975,0.818004,0.706219,0.743836,0.602989,0.742042,0.643735,0.657386,0.740345,...,0.748432,0.547436,0.597274,0.680124,0.692098,0.692626,0.684315,0.769898,0.711369,0.787964
TCGA-78-7155,0.193975,1.000000,0.249829,0.395890,0.204422,0.482311,0.372793,0.508959,0.354464,0.398558,...,0.289107,0.407242,0.377518,0.284280,0.339471,0.346341,0.327364,0.412694,0.285801,0.314715
TCGA-44-7659,0.818004,0.249829,1.000000,0.826650,0.785593,0.700999,0.825551,0.724933,0.821875,0.843355,...,0.665225,0.681551,0.651957,0.820354,0.843185,0.825097,0.810799,0.866557,0.777071,0.865215
TCGA-44-A4SU,0.706219,0.395890,0.826650,1.000000,0.706079,0.803676,0.841651,0.843599,0.752557,0.894144,...,0.749176,0.746985,0.772428,0.845118,0.897244,0.874354,0.873428,0.891760,0.870929,0.887174
TCGA-95-7948,0.743836,0.204422,0.785593,0.706079,1.000000,0.584029,0.734438,0.613603,0.619221,0.777417,...,0.492293,0.505619,0.502785,0.722540,0.716956,0.655740,0.715913,0.721765,0.663815,0.758829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-MP-A4T4,0.692626,0.346341,0.825097,0.874354,0.655740,0.776320,0.878001,0.806946,0.767686,0.819515,...,0.756999,0.717192,0.847667,0.901430,0.848721,1.000000,0.897432,0.839914,0.898881,0.885355
TCGA-78-7160,0.684315,0.327364,0.810799,0.873428,0.715913,0.802789,0.824020,0.772543,0.750039,0.870535,...,0.667334,0.767336,0.776790,0.915711,0.910966,0.897432,1.000000,0.884854,0.884864,0.867522
TCGA-97-A4M3,0.769898,0.412694,0.866557,0.891760,0.721765,0.807358,0.843136,0.830240,0.814988,0.900011,...,0.727136,0.793337,0.726181,0.825672,0.884542,0.839914,0.884854,1.000000,0.861065,0.881172
TCGA-50-6595,0.711369,0.285801,0.777071,0.870929,0.663815,0.780360,0.863865,0.752995,0.728182,0.826681,...,0.779388,0.735896,0.796939,0.877620,0.849621,0.898881,0.884864,0.861065,1.000000,0.894268
